Sweet!
===
01 Gave us a few tools which seem useful in analyzing sound. To encode the sound, lets break up the file into small samples. Small enough that we can focus on only voice/unvoiced/silent utterances, but large enough that low frequencies can be recorded and noticed.

[transfer url](http://stevemorphet.weebly.com/speech-and-audio-processing.html)

First, lets get the imports setup

In [7]:
# Imports
import matplotlib
matplotlib.use('nbagg')

import IPython
from matplotlib.mlab import find
import matplotlib.pyplot as plt

import wave
import struct
import numpy as np
import math
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy.signal import blackmanharris, fftconvolve
from aubio import source, pitch

Next, lets wrap everything up into just a few nice/clean functions

In [5]:
def sampProfile(sig, fs):
    # get RMS
    ms = [(abs(i) ** 2)/float(len(sig)) for i in sig]
    rms = np.sqrt(sum(ms))
    
    # get Zero Crossings
    zero_crossings = 0
    last_val = sig[0]
    for i in range(1,len(sig)):
        if last_val > 0 and sig[i] < 0:
            zero_crossings +=1
        elif last_val < 0 and sig[i] > 0:
            zero_crossings += 1
        last_val = sig[i]
        
    # get Fundamental Frequency
    win_s = len(sig)
    pitch_o = pitch('yin', win_s, win_s, fs)
    sig_for_pitch = np.array(sig).astype(np.float32) #recast type
    fund_freq = pitch_o(sig_for_pitch)[0]
    
    return rms, zero_crossings, fund_freq

In [8]:
samples = ["aaa.wav", "k.wav", "noVoice.wav", "200sine.wav"]


# In seconds:
targetWindow = .2
# In % of sampSize
targetStep = .5



for fil in samples:
    fs, sig = wavfile.read(fil)
    try:
        if len(sig[0]) > 1:
            sig1, sig2 = zip(*sig)
            sig = sig1
            
    except:
        pass
    sampSize = 2**math.ceil(np.log2(fs*targetWindow))
    print sampSize

2048.0
2048.0
2048.0
2048.0
